In [ ]:
# Terraform an RDS instance: rds.tf
# Output the RDS connection props to a config file
# SQL script to make tables in RDS instance
# Python script to pull data, write to file, and copy to RDS instance

In [ ]:
import requests
import json

In [ ]:
API_KEY = ''
BASE_URL = 'https://api.open.fec.gov/v1'
DEFAULT_PARAMS = {
  'per_page': 100
}
ENDPOINTS = {
  'candidates': 'candidates',
  'committees': 'committees',
  'donations': 'schedules/schedule_a'
}

# Keys to pull from the FEC response objects
CANDIDATE_ATTRIBUTES = ['candidate_id', 'name', 'cycles', 'district_number', 'election_districts', 'incumbent_challenge', 'party_full', 'state', 'office_full']
COMMITTEE_ATTRIBUTES = ['committee_id', 'committee_type_full', 'designation_full', 'name', 'organization_type_full', 'state', 'party_full''committee_type_full']
DONATION_ATTRIBUTES = ['contribution_receipt_date', 'contribution_receipt_amount', 'contributor_state', 'contributor_id', 'contributor_name', 'pdf_url', 'sub_id']
session = requests.Session()

In [ ]:
def format_params(param, value):
  return_params = []
  if isinstance(value, list):
    for value_option in value:
      return_params.append('{}={}'.format(param, value_option))
  else:
    return_params.append('{}={}'.format(param, value))
  return return_params

def get_url_params(params):
  default_params = DEFAULT_PARAMS.copy()
  default_params.update(params)
  url_params = []
  for param, value in default_params.items():
    url_params.extend(format_params(param, value))
  return url_params

In [ ]:
def get_url(base_url, endpoint, api_key, **kwargs):
  # Pass kwargs to add params to the URL for filters, etc.
  # See params for your endpoint in the docs: https://api.open.fec.gov/developers/
  url_params = get_url_params(kwargs.copy())
  endpoint = '/'.join([base_url, endpoint])
  query_url = endpoint + '?api_key={}&'.format(api_key) + '&'.join(url_params)
  print(query_url)
  return query_url

In [ ]:
def get_data(endpoint, **kwargs):
  endpoint = ENDPOINTS[endpoint]
  url = get_url(BASE_URL, endpoint, API_KEY, **kwargs)
  response = session.get(url)
  return response.json()

In [ ]:
def format_record(record, keys):
  return {key: value for key, value in record.items() if key in keys}

In [ ]:
def get_object_ids(objects, id_key):
  return list(map(lambda object: object[id_key], objects))

In [ ]:
# Example usage:

# Start with candidates
candidate_data = get_data('candidates', party='DEM', name='CLINTON, HILLARY RODHAM')['results']
candidates = [format_record(candidate, CANDIDATE_ATTRIBUTES) for candidate in candidate_data]
candidates

In [ ]:
# Loop through candidates for Committee data
committees_data = get_data('committees', candidate_id=get_object_ids(candidates, 'candidate_id'))['results']
committees = [format_record(committee, COMMITTEE_ATTRIBUTES) for committee in committees_data]
committees

In [ ]:
# Loop through committees for donation data
donation_data = []
for committee_id in get_object_ids(committees, 'committee_id'):
  donations_subset = [format_record(donation, DONATION_ATTRIBUTES) for donation in get_data('donations', committee_id=committee_id, is_individual="false")['results']]
  donation_data.extend(donations_subset)

In [1]:
# Write data to respective files and trigger copy from local file to RDS
# Write each of these files to S3 for easy access and backup

In [2]:
# Write data to file
def write_to_file(record):
  pass

# Python script to read in the rds config file and run copy command 
def copy_to_rds(file):
  pass

def push_to_s3(file):
  pass

In [ ]:
### IGNORE EVERYTHING BELOW THIS FOR NOW
### JUST GET RAW DATA INTO THE DB USING THE CODE ABOVE. 
### WE CAN NORMALIZE AND UPDATE THE SCHEMA LATER IF WE NEED TO


In [ ]:

def split_running_mates(office, campaign_name):
  candidates = campaign_name.split('/')
  candidate = candidates[0].strip(' ')
  running_mate = candidates[1].strip(' ') if len(candidates) == 2 else 'N/A'
  return (candidate, running_mate)

In [ ]:
def format_candidate(candidate_id, candidate):
    if ',' in candidate:
      names = candidate.split(',')
      print('names: ', names)
      last_name = names[0]
      other_names = names[1].strip(' ').split(' ')
      print('other names:', other_names)      
      first_name = other_names[0]
      if len(other_names) > 1:
        middle_names = ' '.join(other_names[1:])
      else:
        middle_names = ''
    else:
      names = candidate.split(' ')
      first_name = names[0]
      last_name = names[-1]
      middle_names = ' '.join(names[1:-1])
    return {'id': candidate_id, 'first_name': first_name, 'middle_names': middle_names, 'last_name': last_name}
      

In [ ]:
def format_campaign_data(candidate_data):
  campaigns = []
  for record in candidate_data:
    election_years = record.get('election_years', [])
    office = record.get('office_full', 'N/A')
    campaign_name = record.get('name', 'Unknown')
    candidate_id = record.get('candidate_id', '')
    for election_year in election_years:
      campaign = {
        'id': candidate_id + '-' + str(election_year), 
        'campaign_name': campaign_name,
        'office': office
      }
      candidate, running_mate = split_running_mates(office, campaign_name)
      campaign['candidate'] = candidate
      campaign['running_mate'] = running_mate
      campaigns.append(campaign)
  return campaigns  

In [ ]:
# campaigns = format_campaign_data(candidates)
# unique_candidates = set([campaign['candidate'] for campaign in campaigns] + [campaign['running_mate'] for campaign in campaigns if campaign['running_mate'] != 'N/A'])
# labeled_candidates = [format_candidate(index, candidate) for index, candidate in enumerate(unique_candidates)]
# labeled_candidates